house (<u>***house_id*** </u>, name, <u>*host_id* </u>, description, room_type, latitude, longitud, post_time)

host (<u>***host_id***</u>, host_name, about, join_date, location, identity_verified, pic_url)

reviewer (<u>***reviewer_id***</u>, reviewer_name, about, join_date, location, identity_verified, pic_url)

price (<u>**house_id**</u>, **date**, price)

review (<u>**house_id**</u>, <u>**date**</u>, <u>***reviewer_id***</u>, review_name, host_name, score, comment, review_time, rating,reviewer_id, review_profile_path)   






详细爬取方法参考[这篇文章](https://cloud.tencent.com/developer/article/1523321)   
[各种api详细信息](https://stevesie.com/apps/airbnb-api)

### House listing api
https://www.airbnb.com/api/v2/pdp_listing_details/42471710?_format=for_rooms_show&adults=1&key=d306zoyjsyarp7ifhu67rjxn52tv0t20&    

 
### Review detail api
https://airbnb.com/api/v2/reviews?key=d306zoyjsyarp7ifhu67rjxn52tv0t20&locale=en&listing_id=42471710&role=guest&_format=for_p3&_limit=100

###  user(Host/Reviewer) api
https://api.airbnb.com/v2/users/13720227?key=d306zoyjsyarp7ifhu67rjxn52tv0t20&locale=en

### Price api

https://www.airbnb.com/api/v2/pdp_listing_booking_details?_format=for_web_dateless&_intents=p3_book_it&_interaction_type=pageload&_p3_impression_id=p3_1552520120_con%2FMRJw4xlFD6zi&_parent_request_uuid=a23f08cc-f28a-48d8-85e9-c05735e18ff9&currency=USD&force_boost_unc_priority_message_type=&guests=1&key=d306zoyjsyarp7ifhu67rjxn52tv0t20&locale=en&show_smart_promotion=0&checkin=2021-11-04&checkout=2021-11-09&listing_id=42471710

In [40]:
import json
import os
import time
import pickle
import requests
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from lxml import etree
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')

In [36]:
# bro.get('https://www.airbnb.com/s/College-Park--MD/homes?adults=1&place_id=ChIJmUZxDR7Et4kRlW3TbeMNjOs&refinement_paths%5B%5D=%2Fhomes&checkin=2021-11-01&checkout=2021-11-04')
def getOnepage(browser, url):
    '''craw a searching page
    '''
    browser.get(url)
    time.sleep(3)
    soup = BeautifulSoup(browser.page_source,'lxml', fromEncoding = 'ISO-8859-1')
    return browser, browser.page_source, soup

# cityPagesource = bro.page_source
def get_list(soup):
    '''return a house listing from searching page
    '''
    urlList = []
    for i in soup.find_all("div", {"class": "_qlq27g"}):
        url = 'https://airbnb.com/' + i.a['href'] + '&locale=en' # force to request english version
        urlList.append(url)
    return urlList

In [42]:
bro = webdriver.Chrome()
urlList = []
for i in tqdm(range(0,10)):
    search_url = 'https://www.airbnb.com/s/College-Park--MD/homes?adults=1&place_id=ChIJmUZxDR7Et4kRlW3TbeMNjOs&refinement_paths%5B%5D=%2Fhomes&checkin=2021-11-11&checkout=2021-11-16&items_offset={}'.format(i*20)
    # print(search_url)
    _ , page_source, soup = getOnepage(bro, search_url)
    urls = get_list(soup)
    urlList.extend(urls)
bro.close()
print('the length of the url list is {}'.format(len(urlList)))


the length of the url list is 200


In [43]:
urlList

['https://airbnb.com//rooms/53216298?adults=1&check_in=2021-11-11&check_out=2021-11-16&previous_page_section_name=1000&locale=en',
 'https://airbnb.com//rooms/25024597?adults=1&check_in=2021-11-11&check_out=2021-11-16&previous_page_section_name=1000&locale=en',
 'https://airbnb.com//rooms/39465450?adults=1&check_in=2021-11-11&check_out=2021-11-16&previous_page_section_name=1000&locale=en',
 'https://airbnb.com//rooms/32310384?adults=1&check_in=2021-11-11&check_out=2021-11-16&previous_page_section_name=1000&locale=en',
 'https://airbnb.com//rooms/36771553?adults=1&check_in=2021-11-11&check_out=2021-11-16&previous_page_section_name=1000&locale=en',
 'https://airbnb.com//rooms/15341135?adults=1&check_in=2021-11-11&check_out=2021-11-16&previous_page_section_name=1000&locale=en',
 'https://airbnb.com//rooms/13577380?adults=1&check_in=2021-11-11&check_out=2021-11-16&previous_page_section_name=1000&locale=en',
 'https://airbnb.com//rooms/24326005?adults=1&check_in=2021-11-11&check_out=2021-11

In [41]:
urlList

with open('urlList.data', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(urlList, filehandle)

In [ ]:
with open('listfile.data', 'rb') as filehandle:
    urlList = pickle.load(filehandle)

In [19]:
for i in urlList:
    house_id = i.split('airbnb.com//rooms/')[1].split('?adults')[0] # extract house_id from url
    # print(house_id)

51969166
27964129
47472250
28586067
52536246
29474999
42545101
45073061
40280711
32310384
52305136
52244554
22034120
39253525
52509681
50541085
25418737
27133761
51093299
51271095
37295012
43725282
51129838
51364800
52027275
51666017
38481458
44447090
32942690
21203282
38817187
47011083
19363065
49930735
50437389
53067227
23139332
53067886
35285930
3278242
16427522
52371091
49360448
39217552
15610482
51434786
24084817
26601219
30501126
31592816
45712761
40341398
53024904
46231785
23480691
41983676
52851329
16837815
52525678
1472332
49232529
50781156
3627430
50113467
33850755
50938181
51404399
15328533
52108462
27546981
50268247
19767116
16629472
45424729
7714880
51413782
51670924
50337835
22889193
33260758
10787059
53026127
51517345
2318847
36501906
16947075
47585920
46529527
49835875
52410664
52841874
50337849
16954219
52615864
51241902
48977323
21045545
11528631
11317188
32168514
17116424
16310607
47871334
46372366
41718233
45481585
52908157
20604877
11715358
4085972


In [58]:
house_review_json = 'https://www.airbnb.com/api/v2/reviews?key=d306zoyjsyarp7ifhu67rjxn52tv0t20&locale=en&listing_id=16465485&role=guest&_format=for_p3&_limit=100'
response = requests.get(house_review_json)
json_input = response.json()

# review table
def extract_review(json_input):
    '''iterally extract review'''
    reviewDT = pd.DataFrame()
    
    for rev in json_input['reviews']:
        reviewDict = {'reviewer_name':'null',
              'host_name':'null',
              'review_id':'null',
              'comments':'null', 
              'review_time':'null',
              'review_rating':'null', 
              'reviewer_id':'null'}  
        reviewDict['reviewer_name'] = rev['reviewer']['first_name'] # reviewers' first name
        reviewDict['host_name'] = rev['reviewer']['host_name'] # reviewers' first name
        reviewDict['review_id'] = str(rev['id']) # review id
        reviewDict['comments'] = rev['comments'] # comments
        reviewDict['review_time'] = rev['created_at'] # review time
        reviewDict['review_rating'] = rev['rating'] # rating
        reviewDict['reviewer_id'] = int(rev['reviewer']['id']) # reviewer id
        # reviewDict['review_profile_path'] = rev['reviewer']['profile_path'] # profile_path
        # print(reviewDict)
        reviewDT = reviewDT.append(reviewDict, ignore_index=True)
    reviewDT['reviewer_id'] = reviewDT['reviewer_id'].astype('int64')
    return reviewDT

In [59]:
cc = extract_review(json_input)
#cc['review_id'] = cc['review_id'].astype('int64')
cc

,comments,host_name,review_id,review_rating,review_time,reviewer_id,reviewer_name
0,"Nice, relaxing stay! Perfect for our weekend t...",Hope,490535261922211265,5.0,2021-11-07T19:26:00Z,33316698,Hope
1,Lovely home in a safe neighborhood close to ma...,Nick,484767995264996149,5.0,2021-10-30T20:27:28Z,116451079,Nick
2,Brad’s place was great! Very clean and accurat...,Deshon,482572324057177135,5.0,2021-10-27T19:45:03Z,57966510,Deshon
3,Brad is an incredible host. He went above and...,Mary,480377444821634581,5.0,2021-10-24T19:04:13Z,406406843,Mary
4,"Brad's place truly is a ""tranquil oasis."" And ...",Sarah,477444019160375673,5.0,2021-10-20T17:56:02Z,420670971,Sarah
...,...,...,...,...,...,...,...
95,SPEECHLESS ... that’s all I will say.,Tyhirah,642247673,5.0,2020-07-25T18:08:15Z,347605082,Tyhirah
96,:),Kenny,641797813,5.0,2020-07-24T18:42:06Z,123457830,Kenny
97,.,Keeks,640569288,4.0,2020-07-20T19:43:18Z,353463196,Keeks
98,"10/10 place, Brad welcomed us with open arms a...",Michaela,640157095,5.0,2020-07-19T20:10:09Z,325080904,Michaela


In [29]:
listing_json = 'https://www.airbnb.com/api/v2/pdp_listing_details/42471710?_format=for_rooms_show&adults=1&key=d306zoyjsyarp7ifhu67rjxn52tv0t20&'
response = requests.get(listing_json)
listing_json = response.json()
# listing_json

In [34]:
def extract_json_listing(listing_json):
    '''extract listing json
    '''
    listingDict = {'house_id':'null',
              'host_name':'null',
              'host_id':'null',
              'title':'null', 
              'address':'null',
              'latitude':'null', 
              'longtitude':'null', 
              'checkin_rating':'null',
              'clean_rating':'null',
              'comm_rating':'null',
              'satis_overall':'null',
              'bathroom_type':'null',
              'bed_type':'null',
              'bedroom_type':'null',
              'guest_type':'null',
    }
    
    listingDict['house_id'] = listing_json['pdp_listing_detail']['id']
    listingDict['host_name'] = listing_json['pdp_listing_detail']['primary_host']['host_name']
    listingDict['host_id'] = listing_json['pdp_listing_detail']['primary_host']['id']
    listingDict['title'] = listing_json['pdp_listing_detail']['name'] # title
    listingDict['address'] = listing_json['pdp_listing_detail']['p3_summary_address'] # address 
    listingDict['latitude'] = listing_json['pdp_listing_detail']['p3_event_data_logging']['listing_lat'] # lat
    listingDict['longtitude'] = listing_json['pdp_listing_detail']['p3_event_data_logging']['listing_lng'] # lng
    listingDict['checkin_rating'] = listing_json['pdp_listing_detail']['p3_event_data_logging']['checkin_rating'] 
    listingDict['clean_rating'] = listing_json['pdp_listing_detail']['p3_event_data_logging']['cleanliness_rating'] 
    listingDict['comm_rating'] = listing_json['pdp_listing_detail']['p3_event_data_logging']['communication_rating'] 
    listingDict['satis_overall'] = listing_json['pdp_listing_detail']['p3_event_data_logging']['guest_satisfaction_overall']
    # JSON.pdp_listing_detail.bathroom_label
    listingDict['bathroom_type'] = listing_json['pdp_listing_detail']['bathroom_label']
    listingDict['bed_type'] = listing_json['pdp_listing_detail']['bed_label']
    listingDict['bedroom_type'] = listing_json['pdp_listing_detail']['bedroom_label']
    listingDict['guest_type'] = listing_json['pdp_listing_detail']['guest_label']
    return listingDict

extract_json_listing(listing_json)

{'house_id': 42471710,
 'host_name': 'Xin',
 'host_id': 338175337,
 'title': '5 min. Walk Metro. Easy to DC. Big room/bath. 2d',
 'address': 'Hyattsville, Maryland, United States',
 'latitude': 38.96438,
 'longtitude': -76.95835,
 'checkin_rating': 10,
 'clean_rating': 9,
 'comm_rating': 10,
 'satis_overall': 95,
 'bathroom_type': '1 private bath',
 'bed_type': '3 beds',
 'bedroom_type': '1 bedroom',
 'guest_type': '4 guests'}

In [44]:
url = 'https://api.airbnb.com/v2/users/13720227?key=d306zoyjsyarp7ifhu67rjxn52tv0t20&locale=en'
response = requests.get(url)
user_json = response.json()
user_json

# host (host_id, host_name, about, join_date, location, identity_verified, pic_url)
# reviewer (reviewer_id, reviewer_name, about, join_date, location, identity_verified, pic_url)
def extract_user(user_json, reviewer=True):
    '''extract reviewer/host and host information
    '''
    if reviewer:
        userDict = {'reviewer_id':'null',
              'first_name':'null',
              'join_date':'null',
              'about':'null',
              'identity_verified':'null', 
              'picture_url':'null',
              'created_at':'null', 
              'location':'null'
        }
        userDict['reviewer_id'] = user_json['user']['id']
    else:
        userDict = {'host_id':'null',
              'first_name':'null',
              'join_date':'null',
              'about':'null',
              'identity_verified':'null', 
              'picture_url':'null',
              'created_at':'null', 
              'location':'null'
        }
        userDict['host_id'] = user_json['user']['id']
    userDict['first_name'] = user_json['user']['first_name']
    userDict['join_date'] = user_json['user']['created_at']
    userDict['about'] = user_json['user']['about']
    userDict['identity_verified'] = user_json['user']['identity_verified']
    userDict['picture_url'] = user_json['user']['picture_url']
    userDict['created_at'] = user_json['user']['created_at']
    userDict['location'] = user_json['user']['location']
    return userDict
extract_user(user_json)

{'reviewer_id': 13720227,
 'first_name': 'Brad',
 'join_date': '2014-03-31T06:01:53Z',
 'about': "I've lived in The District for nearly 34 years.  I love this great city and all it offers.  Feel free to ask me for recommendations or any other information about DC.  I've lived in this house since 2001 and have enjoyed hosting people from all over the world.",
 'identity_verified': True,
 'picture_url': 'https://a0.muscache.com/im/pictures/user/f4d9817e-6615-4d56-ab4b-4479da564cd6.jpg?aki_policy=profile_x_medium',
 'created_at': '2014-03-31T06:01:53Z',
 'location': 'Washington, DC'}

In [14]:
# Making a get request
#house_review_json = 'https://www.airbnb.com/api/v2/reviews?key=d306zoyjsyarp7ifhu67rjxn52tv0t20&locale=en&listing_id=42471710&role=guest&_format=for_p3&_limit=100'
#response = requests.get(house_review_json)
# print json content
#print()
#house_review_json = 'https://www.airbnb.com/api/v2/reviews?key=d306zoyjsyarp7ifhu67rjxn52tv0t20&locale=en&listing_id=16465485&role=guest&_format=for_p3&_limit=100'
#response = requests.get(house_review_json)
#response.json()

## response.json()['reviews'][3]['comments']
## listing-42471710 > div > div._ni9axhe > div > div._e296pg > div > div > div > a
## soup = BeautifulSoup(cityPagesource,'lxml', fromEncoding = 'ISO-8859-1')
## url.split('airbnb.com//rooms/')[1].split('?adults')[0] # 爬取room_id
## 
## soup.title.string.split('-')[0] # 爬取title
## soup.title.string.split('-')[1] # 爬取地点
## soup.title.string.split('-')[1].split(',')[-2].strip() # 州
## soup.title.string.split('-')[1].split(',')[-1].strip() # 国
## soup.title.string.split('-')[1].split(',')[-3].split('in')[-1].strip() # city
## 
## soup.find_all("a", {"class": "_105023be"})[0]['href'] # host id
## 
## list1 = []
## for i in soup.find_all("ol", {"class": "_194e2vt2"}):
##     list1.extend([i.string for i in i.find_all('span')])  # 
##     #url =  i.a['href'] # 强行改为英文网页
##     #print(url)
## list1
## soup.find_all("a", {"class": "_105023be"})[0]['href']
## Host information
## for i in soup.find_all("span", {"class": "_1ax9t0a"}):
##   print(i.get_text())
## soup.find_all("h1", {"class": "_14i3z6h"})[1].get_text() # host name
## soup.find_all("div", {"class": "_u5okn1"})[0].get_text() # join date
## soup.find_all("span", {"class": "_1ax9t0a"})[0].get_text() # reviews number
## soup.find_all("span", {"class": "_1ax9t0a"})[1].get_text() # Identity Verification
## soup.find_all("span", {"class": "_1ax9t0a"})[6].get_text() # address 
## soup.find_all("span", {"class": "_1ax9t0a"})[4].get_text()
## soup.find_all("div", {"class": "_u5okn1"})[0].get_text()
## soup.find_all("div", {"class": "_u5okn1"})[0].get_text()
## response.json()['reviews'][4]['comments']